In [2]:
import pyclifford as pc
import numpy as np

## Pauli operator

### Construct Pauli operator with its expression

A **Pauli operator** can be constructed using the `pauli()` constructor.

In [3]:
pc.pauli('XXIYZ')

 +XXIYZ

By default the operator has a $+1$ phase factor in the front. To specify other phase factors($\pm1$ or $\pm \mathrm{i}$), use `'+'`, `'-'`, `'i'` indicators before the Pauli string.

In [6]:
pc.pauli('-X'), pc.pauli('iX'), pc.pauli('-iX')

( -X, +iX, -iX)

It is also possible to assign the phase factor by scalar mutiplication.

In [7]:
-pc.pauli('X'), 1j*pc.pauli('X')

( -X, +iX)

### Other methods to construct a Pauli operator

You can construct a Pauli operator from a tuple / list / array of indices (`0` = `I`, `1` = `X`, `2` = `Y`, `3` = `Z`)

In [9]:
pc.pauli((0,1,2,3)), pc.pauli([0,1,2,3]), pc.pauli(np.array([0,1,2,3]))

( +IXYZ,  +IXYZ,  +IXYZ)

Or you can construct a Pauli operator from a dictionary that maps positions to indices. (*Note*: using this method must also provide the total number of qubits as the second argument, because the qubit number can not be infered from the dictionary alone.)

In [10]:
pc.pauli({1:'X', 4:'Y', 5:'Z'}, 6), pc.pauli({1:1, 4:2, 5:3}, 6)

( +IXIIYZ,  +IXIIYZ)

### Size information

For Pauli operator, `.N` returns the number of qubits (size of system) that the operator acts on.

In [11]:
pc.pauli('IXYIXI').N

6

## Pauli operator list

A **list of Pauli operators** can be constructed by the `paulis()` constructor.

In [12]:
pc.paulis('iX', '-iY', 'Z')

+iX
-iY
 +Z

It can also take a generator and iterate through its elements to construct a list of Pauli operators.

In [13]:
pc.paulis(pc.pauli({i:'Z'}, 4) for i in range(4))

 +ZIII
 +IZII
 +IIZI
 +IIIZ

It can also take a iterable (tuple / list / set) and convert it to a list of Pauli operators.

In [14]:
lists = ['iX', '-iY', 'Z']
pc.paulis(lists)

+iX
-iY
 +Z

### Size information

For Pauli operator list, `.L` returns the number of operators in the list and `.N` returns of the number fo qubits in the system.

In [15]:
plst = pc.paulis('II','XX','YY','ZZ')
plst.L, plst.N

(4, 2)

We can also return the number of operators in the list by naive python `len()` function

In [16]:
len(plst)

4

### Selection and Slicing of Pauli List

Select a single element in the Pauli operator list.

In [17]:
plst = pc.paulis('II','XX','YY','ZZ')
plst[1]

 +XX

Select a range of operators in the Pauli operator list: the slicing is the same as python array

In [18]:
plst[0:3]

 +II
 +XX
 +YY

It is also allow to be selected by a index array or a boolean mask.

In [19]:
plst[np.array([2,1,1,0,3])]

 +YY
 +XX
 +XX
 +II
 +ZZ

In [20]:
plst[np.array([True,False,False,True])]

 +II
 +ZZ

## Pauli Polynomials

Pauli operators can be linearly combined in to a **Pauli polynomial**.

In [22]:
pc.pauli('XX') + pc.pauli('YY') - 0.5 * pc.pauli('ZZ')

-0.50 ZZ +1 XX +1 YY

<div class="alert alert-block alert-success">
Adding Pauli operators with any number, the number will be promoted to the number times identity operator automatically. For example, a projection operator can be written as
</div>

In [23]:
(pc.pauli('ZZ') + 1)/2

0.50 II +0.50 ZZ

Operators can be summed up with python built-in function `sum()`.

In [25]:
sum(pc.paulis('II','XX','YY','ZZ'))

1 II +1 ZZ +1 XX +1 YY

## Pauli Algebra

### Dot Productor (Matrix Multiplication)

Dot productor (composition) of Pauli operators is implemented as the matrix multiplication `matmul`, which can be implemented using the operand `@`.

In [26]:
pc.pauli('X') @ pc.pauli('Y'), pc.pauli('Y') @ pc.pauli('X')

(+iZ, -iZ)

Dot product of Pauli polynomials will be expanded.

In [27]:
poly = pc.pauli('XX') + pc.pauli('YY') - 0.5 * pc.pauli('ZZ')
poly @ poly

0.25 II +0.50 YY +0.50 XX +0.50 YY +1 II -1 ZZ +0.50 XX -1 ZZ +1 II

Terms will not be combined automatically. To combine them, the `.reduce()` method should be explicitly called.

In [28]:
(poly @ poly).reduce()

2.25 II -2 ZZ +1 XX +1 YY

### Trace of Pauli operators

- `Pauli.trace()` will return the trace of a Pauli operator
 - `PauliList.trace()` will return the trace of a list of Pauli operators
 - `PauliPolynomial.trace()` will return the trace of a Pauli polynomial

In [30]:
(3*pc.pauli('II')+(3+2.5j)*pc.pauli('II')).trace()

(24+10j)

### Weight (number of non-identity support)

In [31]:
pc.pauli('IXIYZII').weight()

3

In [32]:
pc.paulis('IXIYZII','IXIIIII').weight()

array([3, 1])

### Type conversion

Automatic type conversion enables the algebra to be carried out among different classes with great flexibiliity.
* When `Pauli` is multiplied (`*`) by a generic number (beyond powers of the imaginary unit), it is converted to `PauliMonomial`.
* When `Pauli` or `PauliMonomial` is added (`+`) or subtracted (`-`) with other Pauli objects, they are converted to `PauliPolynomial`.
* The dot product (`@`) generally returns `PauliPolynomial`, unless the two Pauli objects are both `Pauli`, in which case it returns `Pauli`.

## Clifford Transformation

 `PauliList` provides useful methods to implement Clifford transformations efficiently on all Pauli operators together. The same methods are available to all its subclasses (including `PauliPolynomial`, `CliffordMap`, `StabilizerState`).

### Clifford Rotation

A Clifford rotation is a $\mathbb{Z}_4$ rotation in the Clifford group generated by a single Pauli operator, which takes the form of
$$
U=e^{\frac{i\pi}{4}\sigma}=\frac{1}{\sqrt{2}}(1+i \sigma)
$$
Every Pauli operator is transformed by $\sigma \to U^\dagger \sigma U$. The Clifford rotation can be applied by the method `.rotate_by(gen)` (given the generator `gen`). The operation is in-place (meaning that the operators in the Pauli list will be modified).

In [33]:
pc.paulis('II','XX','YY','ZZ').rotate_by(pc.pauli('XI'))

 +II
 +XX
 +ZY
 -YZ

### Clifford Map

A Clifford map is a generic clifford transformation by specifying how each single Pauli operator gets mapped to. It can be listed as a table

In [36]:
cmap = pc.random_clifford_map(2)
cmap

CliffordMap(
  X0-> +ZX
  Z0-> -ZZ
  X1-> +YY
  Z1-> +XY)

It can be applied by the method `.transform_by(cmap)` (given the Clifford map `cmap`). 

In [37]:
pc.paulis('II','XX','YY','ZZ').transform_by(cmap)

 +II
 +XZ
 -ZY
 -YX

### Masked Transformation

Clifford transformation can be applied to a subsystem of qubits specified by a mask.

In [38]:
mask = np.array([True,False,False,True])
pc.paulis('IIII','XXXX','YYYY','ZZZZ').rotate_by(pc.pauli('XY'), mask)

 +IIII
 -IXXZ
 +ZYYI
 +ZZZZ

In [39]:
mask = np.array([True,False,False,True])
pc.paulis('IIII','XXXX','YYYY','ZZZZ').transform_by(cmap, mask)

 +IIII
 +XXXZ
 -ZYYY
 -YZZX

## Export to Numpy Array

Pauli operators can be easily converted to the `Qobj` in qutip library
- `Pauli.to_numpy()`
- `PauliList.to_numpy()`
- `PauliMonomial.to_numpy()`
- `PauliPolynomial.to_numpy()`

In [4]:
pc.pauli('XY').to_numpy()

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
       [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j],
       [0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [5]:
pc.paulis('XY','ZZ').to_numpy()

array([[[ 0.+0.j,  0.+0.j,  0.+0.j,  0.-1.j],
        [ 0.+0.j,  0.+0.j,  0.+1.j,  0.+0.j],
        [ 0.+0.j,  0.-1.j,  0.+0.j,  0.+0.j],
        [ 0.+1.j,  0.+0.j,  0.+0.j,  0.+0.j]],

       [[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j,  0.+0.j, -0.+0.j],
        [ 0.+0.j,  0.+0.j, -1.+0.j, -0.+0.j],
        [ 0.+0.j, -0.+0.j, -0.+0.j,  1.+0.j]]])

In [6]:
((2+3j)*pc.pauli('XY')).to_numpy()

array([[ 0.+0.j,  0.+0.j,  0.+0.j,  3.-2.j],
       [ 0.+0.j,  0.+0.j, -3.+2.j,  0.+0.j],
       [ 0.+0.j,  3.-2.j,  0.+0.j,  0.+0.j],
       [-3.+2.j,  0.+0.j,  0.+0.j,  0.+0.j]])